# Project 4: Decision Trees

### Name: David Hill
### Course Level: CSC 448

**Introduction:**
* In this project, we explore the application of classification using: Decision Tree

**Objectives:**
* The objective of this project is to develop software modules for classification of Titanic passengers via decision trees

## All Students

* The first problem we aim to us Information Gain to build a decision tree for classification.  The tree will complete when one of two conditions are met:

1. The maximum tree depth is reached.
2. The minimum number of samples is reached in a given leaf node.

**Problem A (100pts)**

1 (5pts). The first thing you'll need to do is read the Titatinic dataset from D2L (details about the dataset can be found [Here](https://www.kaggle.com/c/titanic/data?select=test.csv)).  Note that you will need both the train.csv (to build your tree) and test.csv to evaluate the classificaiton accuracy.

In [21]:
# Read in the data (however you prefer) #
import pandas as pd
import numpy as np

# read in training data
df = pd.read_csv('train.csv')
d_data = df.drop(['PassengerId', 'Survived', 'Name', 'SibSp', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1) # throw out irrelevant columns
d_data['Sex'] = d_data['Sex'].map({'female': 0, 'male': 1}) # make sex column numeric values
d_data.fillna(d_data.median(), inplace=True) # fill missing values with median
d_data = d_data.values # convert to numpy arrays
l_labels = df['Survived'].values # only take survived column for labels

# read in testing data
test_df = pd.read_csv('test.csv')
test_data = test_df.drop(['PassengerId', 'Survived', 'Name', 'SibSp', 'Ticket', 'Fare', 'Cabin', 'Embarked'], axis=1)
test_data['Sex'] = test_data['Sex'].map({'female': 0, 'male': 1})
test_data.fillna(test_data.median(), inplace=True)
test_data = test_data.values
test_labels = test_df['Survived'].values


2 (30pts). Next, we need a couple helper functions to compute the conditional entropy and information gain.

**Recall: Entropy** 
$$
    H(X) = -\sum_{\textbf{x} \in X} p(\textbf{x}) \log_2( p(\textbf{x}) )
$$

**Recall: Conditional Entropy** 
$$
    H(Y|X=\textbf{x}) = -\sum_{\textbf{y} \in Y} p(\textbf{y}|\textbf{x}) \log_2 ( p(\textbf{y}|\textbf{x}) )
$$
where
$$
    p(\textbf{y}|\textbf{x}) = \frac{p(\textbf{y},\textbf{x})}{p(\textbf{x})}
$$
and
$$
    p(\textbf{x}) = \sum_{\textbf{y} \in Y} p(\textbf{y},\textbf{x})
$$

**Recall: Information Gain** 
$$
    IG(Y|X) = H(Y) - H(Y|X)
$$



In [22]:
# Function definitions here (you may want to modify the input parameters depending on your implimentation #
def Entropy(labels):
    unique_labels, label_counts = np.unique(labels, return_counts=True)
    probabilities = label_counts / len(labels)
    H = -np.sum(probabilities * np.log2(probabilities))
    return H

def C_Entropy(feature_values, labels):
    cH = 0
    unique_values, label_counts = np.unique(feature_values, return_counts=True)
    for value, count in zip(unique_values, label_counts):
        probabilities = np.bincount(labels[feature_values == value]) / count # p(y|x)
        probabilities = np.clip(probabilities, 1e-15, 1.0 - 1e-15)  # avoid zero probability error
        cH += (count / len(feature_values)) * -np.sum(probabilities * np.log2(probabilities))
    return cH

def IG(labels, feature_values, left_indices, right_indices):
    H = Entropy(labels)
    cH = 0
    # calculate weighted average
    for indices in (left_indices, right_indices):
        if len(indices) == 0:
            continue
        cH += (len(indices) / len(labels)) * C_Entropy(feature_values[indices], labels[indices])
    IG = H - cH
    return IG

3 (40pts). Define a function to build the tree, recall there should be one of two exit criterion:

1. The maximum tree depth is reached
2. The minimum number of samples is reached in a given leaf node.


In [23]:
# Build Tree (can be recursive) #
# using OOP
class TreeNode:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.feature_index = 0
        self.threshold = 0
        self.left = None
        self.right = None

def BuildTree(d_data, l_labels, max_depth=None, min_samples_leaf=1, depth=0):
    # check if all labels are the same or if max depth is reached or if the data shrinks underneath the minimum samples per leaf
    if len(np.unique(l_labels)) == 1 or (max_depth is not None and depth == max_depth) or len(d_data) < min_samples_leaf:
        return TreeNode(predicted_class=np.argmax(np.bincount(l_labels)))
    
    best_feature = None
    best_threshold = None
    best_IG = -float('inf') # initialize to -infinity

    for feature_index in range(d_data.shape[1]):
        thresholds = np.unique(d_data[:, feature_index])
        for threshold in thresholds:
            # get indices based on threshold
            left_indices = np.where(d_data[:, feature_index] <= threshold)[0]
            right_indices = np.where(d_data[:, feature_index] > threshold)[0]
            # prevent split depending on samples parameter
            if len(left_indices) < min_samples_leaf or len(right_indices) < min_samples_leaf:
                continue
            ig = IG(l_labels, d_data[:, feature_index], left_indices, right_indices)
            # change "best" variables according to IG
            if ig > best_IG:
                best_IG = ig
                best_feature = feature_index
                best_threshold = threshold
    
    # if best_IG didn't change from initialization
    if best_IG == -float('inf'):
        return TreeNode(predicted_class=np.argmax(np.bincount(l_labels)))

    # get indices based on BEST threshold now
    left_indices = np.where(d_data[:, best_feature] <= best_threshold)[0]
    right_indices = np.where(d_data[:, best_feature] > best_threshold)[0]

    # build left and right trees recursively and increase depth
    left_tree = BuildTree(d_data[left_indices], l_labels[left_indices], max_depth, min_samples_leaf, depth + 1)
    right_tree = BuildTree(d_data[right_indices], l_labels[right_indices], max_depth, min_samples_leaf, depth + 1)

    # get node values and return it
    node = TreeNode(predicted_class=None)
    node.feature_index = best_feature
    node.threshold = best_threshold
    node.left = left_tree
    node.right = right_tree

    return node

4 (20pts). Next write a function to perform the prediciton (classificaiton) of a new training sample:

In [24]:
# Function to predict #
def DT_predict(d_tree,d_sample):
    # traverse the tree until a leaf node is reached
    while d_tree.predicted_class is None:
        if d_sample[d_tree.feature_index] <= d_tree.threshold:
            d_tree = d_tree.left
        else:
            d_tree = d_tree.right
    return d_tree.predicted_class

5 (5pts). Using the test set, evaluate the accuracy of your classification tree.

In [25]:
# Classification Accuracy #
# using test data and test labels read in above

# build tree
tree = BuildTree(d_data, l_labels, max_depth=3, min_samples_leaf=10)

# calculate correct predictions out of length of the test data
correct_predictions = 0
for i in range(len(test_data)):
    sample = test_data[i]
    actual_result = test_labels[i]
    prediction = DT_predict(tree, sample)
    if prediction == actual_result:
        correct_predictions += 1
acc = correct_predictions / len(test_data)

print("Accuracy:", acc)

Accuracy: 0.7679425837320574
